In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import datetime
import numpy as np

In [7]:
start = time.time()

years = list(np.arange(2000,2021,1))
cals = [2] # 1 for WT, 2 for UCI, etc.
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
results_dict = {}
stages_dict = {}

for year in years:
    for month in months:
        for cal in cals:
            url = 'https://firstcycling.com/race.php?y='+str(year)+'&t='+str(cal)+'&m='+str(month)
            r = requests.get(url)
            soup = BeautifulSoup(r.text,'html.parser')

            if year == datetime.datetime.today().year and month == datetime.datetime.today().month: 
                try:
                    tab = soup.find_all('table', class_='sortTabell tablesorter')[1]
                except:
                    continue
            else:
                try:
                    tab = soup.find_all('table', class_='sortTabell tablesorter')[0]
                except:
                    continue

            refs = [i['href'] for i in tab.find_all('a', href=True)]
            races_raw = [x for x in refs if 'race' in x]
            races = [z for z in races_raw if 'k' not in z]
            urls = ['https://firstcycling.com/'+y for y in races]
            for race in urls:
                r_race = requests.get(race)
                soup_race = BeautifulSoup(r_race.text,'html.parser')
                race_name = soup_race.find('h1').text
                rows = soup_race.find_all('tbody')[0].find_all('tr')
                result = []
                for tr in rows:
                    td = tr.find_all('td')
                    row = [tr.text.strip() for tr in td if tr.text.strip()]
                    if row:
                        result.append(row)

                stage_soup = BeautifulSoup(requests.get(race+'&k=2').text,'html.parser') 
                try:
                    if stage_soup.find('h1', class_='delinfo').text == 'Stages':
                        stg_ref = [i['href'] for i in stage_soup.find_all('table', class_="tablesorter")[1].find_all('a')]
                        stages = [x for x in stg_ref if 'race' in x]
                        stages_url = ['https://firstcycling.com/'+y for y in stages]
                        for stage in stages_url:
                            r_stage = requests.get(stage)
                            soup_stage = BeautifulSoup(r_stage.text,'html.parser')
                            for i in soup_stage.find_all('select')[1].find_all('option'):
                                if i.has_attr('selected'):
                                    stage_name = race_name+' stage '+i.text
                            rows_stage = soup_stage.find_all('tbody')[0].find_all('tr')
                            result_stage = []
                            for tr in rows_stage:
                                td = tr.find_all('td')
                                row = [tr.text.strip() for tr in td if tr.text.strip()]
                                if row:
                                    result_stage.append(row)
                            stages_dict[stage_name] = pd.DataFrame(result_stage,
                                                                   columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
                except:
                    continue


                try:        
                    results_dict[race_name] = pd.DataFrame(result, 
                                                           columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
                except ValueError:
                    continue

        print('loop towards month', month, year, 'ended')

end = time.time()
print('elapsed time: '+str((end-start)/60)+' minutes')

loop towards month 01 2000 ended
loop towards month 02 2000 ended
loop towards month 03 2000 ended
loop towards month 04 2000 ended
loop towards month 05 2000 ended
loop towards month 06 2000 ended
loop towards month 07 2000 ended
loop towards month 08 2000 ended
loop towards month 09 2000 ended
loop towards month 10 2000 ended
loop towards month 11 2000 ended
loop towards month 12 2000 ended
elapsed time: 9.990360808372497 minutes
loop towards month 01 2001 ended
loop towards month 02 2001 ended
loop towards month 03 2001 ended
loop towards month 04 2001 ended
loop towards month 05 2001 ended
loop towards month 06 2001 ended
loop towards month 07 2001 ended
loop towards month 08 2001 ended
loop towards month 09 2001 ended
loop towards month 10 2001 ended
loop towards month 11 2001 ended
loop towards month 12 2001 ended
elapsed time: 22.83942241271337 minutes
loop towards month 01 2002 ended
loop towards month 02 2002 ended
loop towards month 03 2002 ended
loop towards month 04 2002 en

loop towards month 12 2018 ended
elapsed time: 329.7487700303396 minutes
loop towards month 01 2019 ended
loop towards month 02 2019 ended
loop towards month 03 2019 ended
loop towards month 04 2019 ended
loop towards month 05 2019 ended
loop towards month 06 2019 ended
loop towards month 07 2019 ended
loop towards month 08 2019 ended
loop towards month 09 2019 ended
loop towards month 10 2019 ended
loop towards month 11 2019 ended
loop towards month 12 2019 ended
elapsed time: 378.37614884376524 minutes
loop towards month 01 2020 ended
loop towards month 02 2020 ended
loop towards month 03 2020 ended
loop towards month 04 2020 ended
loop towards month 05 2020 ended
loop towards month 06 2020 ended
loop towards month 07 2020 ended
loop towards month 08 2020 ended
loop towards month 09 2020 ended
loop towards month 10 2020 ended
loop towards month 11 2020 ended
loop towards month 12 2020 ended
elapsed time: 404.21306645472845 minutes


In [8]:
full_dicts = {**results_dict, **stages_dict}

In [19]:
import pickle

with open('results.p', 'wb') as fp:
    pickle.dump(full_dicts, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
final_df = pd.DataFrame()
for key, value in full_dicts.items():
    df = value
    df.loc[:,'race'] = key
    final_df = pd.concat([df, final_df], 0)
final_df

,Position,Rider,Born,Team,Total time,FC points,race
0,01,Efren Santos,1992,Canel´s Pro Cycling,02:04:37,7,Vuelta a Ecuador - 2020 stage 06 : Quito - Quito
1,02,Santiago Montenegro,1998,Ecuador,+ 35,3,Vuelta a Ecuador - 2020 stage 06 : Quito - Quito
2,03,Joel Burbano,1989,Sin Fronteras Team JN,+ 37,1,Vuelta a Ecuador - 2020 stage 06 : Quito - Quito
3,04,Segundo Navarrete,1985,Movistar Team Ecuador,+ 40,None,Vuelta a Ecuador - 2020 stage 06 : Quito - Quito
4,05,Santiago Ordoñez,1997,Canel´s Pro Cycling,+ 42,None,Vuelta a Ecuador - 2020 stage 06 : Quito - Quito
...,...,...,...,...,...,...,...
96,DNF,Jose Mendez,1999,SuBicicleta.com-Osorio Motos,None,None,Vuelta al Táchira - 2018
97,DNF,Juan Felipe Guzman Achury,1997,Trepa Bikes,None,None,Vuelta al Táchira - 2018
98,DNF,Manuel Enrique Jimenez Silva,1993,Trepa Bikes,None,None,Vuelta al Táchira - 2018
99,DNF,Yerzon Sanchez,1977,Trepa Bikes,None,None,Vuelta al Táchira - 2018


In [20]:
final_df.to_csv('results.csv')

In [17]:
final_df[final_df['race'].isin(['2000'])]

,Position,Rider,Born,Team,Total time,FC points,race


In [50]:
final_df[final_df['race']=='Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo']

,Position,Rider,Born,Team,Total time,FC points,race
0,01,Jakub Mareczko,1994,Vini Zabú,02:19:05,14,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
1,02,Mark Cavendish,1985,Deceuninck-Quick Step,+ 00,5,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
2,03,Marius Mayrhofer,2000,Team DSM,+ 00,3,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
3,04,Luca Coati,1999,Team Qhubeka,+ 00,None,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
4,05,Ethan Hayter,1998,INEOS Grenadiers,+ 00,None,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
...,...,...,...,...,...,...,...
166,167,Mikel Aristi,1993,Euskaltel-Euskadi,+ 00,None,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
167,168,Antonio Angulo,1992,Euskaltel-Euskadi,+ 00,None,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
168,DNS,Gianni Moscon,1994,INEOS Grenadiers,None,None,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
169,DNF,Ibai Azurmendi,1996,Euskaltel-Euskadi,None,None,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo


In [ ]:
temp = results_dict['Paris - Nice - 2021']['Total time'] ##take the total time of the race (it is a string)

temp_list = []
for x in temp:
    if x is not None:
        y = x.split(':')                            ## split it from HH:MM:SS to ['HH', 'MM', 'SS'] (it is a list of strings)
        for i in range(0, len(y)):
            y[i]=int(y[i].replace("+ ", ""))        ## deletes '+ ' from the time gaps
        temp_int = 0
        for j in range(len(y)-1, -1, -1):
            temp_int += y[j]*(60 **(len(y)-(j+1)))  ## converts timegaps into seconds 
        temp_list.append(temp_int)                  ## appends them to list

for k in range(len(temp_list), len(temp)):
    temp_list.append('None')                        ## appends 'None' to list before merging into DF
results_dict['Paris - Nice - 2021']['Time seconds'] = temp_list

results_dict['Paris - Nice - 2021']['Perc of time'] = ""  ## creates empty column for percentage of time

for index in range(1, len(results_dict['Paris - Nice - 2021']['Time seconds'])):      ## sums timegaps and total time of the winner
    if results_dict['Paris - Nice - 2021']['Time seconds'][index] != 'None':
        results_dict['Paris - Nice - 2021']['Time seconds'][index] +=  int(results_dict['Paris - Nice - 2021']['Time seconds'][0])

for index  in range(len(results_dict['Paris - Nice - 2021']['Time seconds'])) :
    if results_dict['Paris - Nice - 2021']['Time seconds'][index] != 'None':
        results_dict['Paris - Nice - 2021']['Perc of time'][index] = int(results_dict['Paris - Nice - 2021']['Time seconds'][index]) / int(results_dict['Paris - Nice - 2021']['Time seconds'][0])
    else:
        results_dict['Paris - Nice - 2021']['Perc of time'][index] = 'None'

    
print(results_dict['Paris - Nice - 2021'])

In [ ]:
for name in results_dict.keys():
    temp = results_dict[name]['Total time'] ##take the total time of the race (it is a string)

    temp_list = []
    for x in temp:
        if x is not None:
            y = x.split(':')                            ## split it from HH:MM:SS to ['HH', 'MM', 'SS'] (it is a list of strings)
            for i in range(0, len(y)):
                y[i]=int(y[i].replace("+ ", ""))        ## deletes '+ ' from the time gaps
            temp_int = 0
            for j in range(len(y)-1, -1, -1):
                temp_int += y[j]*(60 **(len(y)-(j+1)))  ## converts timegaps into seconds 
            temp_list.append(temp_int)                  ## appends them to list

    for k in range(len(temp_list), len(temp)):
        temp_list.append('None')                        ## appends 'None' to list before merging into DF
    results_dict[name]['Time seconds'] = temp_list

    results_dict[name]['Perc of time'] = ""  ## creates empty column for percentage of time

    for index in range(1, len(results_dict[name]['Time seconds'])):      ## sums timegaps and total time of the winner
        if results_dict[name]['Time seconds'][index] != 'None':
            results_dict[name]['Time seconds'][index] +=  int(results_dict[name]['Time seconds'][0])

    for index  in range(len(results_dict[name]['Time seconds'])) :
        if results_dict[name]['Time seconds'][index] != 'None':
            results_dict[name]['Perc of time'][index] = int(results_dict[name]['Time seconds'][index]) / int(results_dict[name]['Time seconds'][0])
        else:
            results_dict[name]['Perc of time'][index] = 'None'

        
    print(results_dict[name])

In [ ]:
for name in stages_dict.keys():
    temp = stages_dict[name]['Total time'] ##take the total time of the race (it is a string)

    temp_list = []
    for x in temp:
        if x is not None:
            y = x.split(':')                            ## split it from HH:MM:SS to ['HH', 'MM', 'SS'] (it is a list of strings)
            for i in range(0, len(y)):
                y[i]=int(y[i].replace("+ ", ""))        ## deletes '+ ' from the time gaps
            temp_int = 0
            for j in range(len(y)-1, -1, -1):
                temp_int += y[j]*(60 **(len(y)-(j+1)))  ## converts timegaps into seconds 
            temp_list.append(temp_int)                  ## appends them to list

    for k in range(len(temp_list), len(temp)):
        temp_list.append('None')                        ## appends 'None' to list before merging into DF
    stages_dict[name]['Time seconds'] = temp_list

    stages_dict[name]['Perc of time'] = ""  ## creates empty column for percentage of time

    for index in range(1, len(stages_dict[name]['Time seconds'])):      ## sums timegaps and total time of the winner
        if stages_dict[name]['Time seconds'][index] != 'None':
            stages_dict[name]['Time seconds'][index] +=  int(stages_dict[name]['Time seconds'][0])

    for index  in range(len(stages_dict[name]['Time seconds'])) :
        if stages_dict[name]['Time seconds'][index] != 'None':
            stages_dict[name]['Perc of time'][index] = int(stages_dict[name]['Time seconds'][index]) / int(stages_dict[name]['Time seconds'][0])
        else:
            stages_dict[name]['Perc of time'][index] = 'None'

        
    print(stages_dict[name])

In [9]:
years = [2020]
cals = [2] # 1 for WT, 2 for UCI, etc.
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
results_dict = {}
stages_dict = {}

for year in years:
    for month in months:
        for cal in cals:
            url = 'https://firstcycling.com/race.php?y='+str(year)+'&t='+str(cal)+'&m='+str(month)
            r = requests.get(url)
            soup = BeautifulSoup(r.text,'html.parser')
            print(url)


https://firstcycling.com/race.php?y=2020&t=2&m=01
https://firstcycling.com/race.php?y=2020&t=2&m=02
https://firstcycling.com/race.php?y=2020&t=2&m=03
https://firstcycling.com/race.php?y=2020&t=2&m=04
https://firstcycling.com/race.php?y=2020&t=2&m=05
https://firstcycling.com/race.php?y=2020&t=2&m=06
https://firstcycling.com/race.php?y=2020&t=2&m=07
https://firstcycling.com/race.php?y=2020&t=2&m=08
https://firstcycling.com/race.php?y=2020&t=2&m=09
https://firstcycling.com/race.php?y=2020&t=2&m=10
https://firstcycling.com/race.php?y=2020&t=2&m=11
https://firstcycling.com/race.php?y=2020&t=2&m=12
